Analyse des données

Préparation des données avec pandas


In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from cleaner import DataCleaner
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from tools import (
    logging,
    bins_generator,
    import_datasets,
    decode_clean,

)
import eda_movies as eda

In [ ]:
link = "clean_datasets/movies_cleaned.parquet"
df = pd.read_parquet(link)
df

In [ ]:
link = "big_dataframe/movies.parquet"
df = eda.eda_movies(link)

ADD TMDB
AND
AKAS

In [ ]:

line = "movies_datasets/tmdb_full.csv"
df_imdb = pd.read_csv(line)

#  Je merge les deux bd
merged = pd.merge(
    df,
    df_imdb,
    left_on="titre_id",
    right_on="imdb_id",
    how="left"
)
logging.info(f"Cleaned : {len(df) - len(merged)} rows")

#  Je drop les colonnes
merged = merged.drop(eda.columns_to_drop_tmdb(), axis=1)
max_ = merged.isna().sum()
logging.info(f"Cleaned NaN Value : {max_.max()}")

merged = merged.dropna()
logging.info(f"Length dataframe merged with tmdb : {len(merged)}")


col_list = ["spoken_languages", "production_countries"]
merged = eda.clean_square_brackets(
    merged,
    col_list
)
merged = merged.dropna()
logging.info(f"Length dataframe merged cleaned : {len(merged)}")

merged = eda.apply_decode_and_split(
    merged,
    col_list,
    decode_clean
)

# Importation Akas !
akas = import_datasets(
    "movies_datasets/title_akas.tsv",
    types="pandas",
    sep="\t"
)

# https://fr.wikipedia.org/wiki/D%C3%A9mographie_de_la_Creuse
# https://fr.statista.com/statistiques/498200/preference-films-etrangers-vo-vf-france/

akas = akas[akas["region"] == 'FR']
region_only = akas[["titleId", "region"]]

#  Merge de akas avec la base de donnée merged (df movies inclus)
logging.info("Merging tmdb and akas dataframes...")
added_region = pd.merge(
    merged,
    region_only,
    left_on="titre_id",
    right_on="titleId"
)
df = added_region

logging.info("Drop all duplicated movies...")
df.drop_duplicates(subset=["titre_id"], keep="first", inplace=True)

# Conserver les films realeased uinqiuement.
condi = (
    df["status"] == "Released"
)
df = df[condi]

In [ ]:
df = df.copy()
df = eda.split_columns(df, "titre_genres")
# logging.info(f"Top most viewed genre by movies : {eda.get_top_genres(df)}")
df = eda.apply_decade_column(df)
df = eda.drop_nan_values(df)

In [ ]:
eda.show_total_films_decade(df)

**A partir d'ici nous pouvons definir notre angle d'attaque**
=


**Delete les films en dessous de 1960**

In [ ]:
condi =(
    (df["titre_date_sortie"] >= 1960) &
    (df["rating_avg"] >= 5)
)
df = df[condi].reset_index(drop='index')

In [ ]:
eda.show_total_films_decade(df)

In [ ]:
plt.figure(figsize=(10, 6))
plt.boxplot(
    df["titre_duree"],
    patch_artist=True
)
plt.title("Durée des films en minutes")
plt.tight_layout()
plt.show()

In [ ]:
condi =(
    (df["titre_duree"] < 60) | (df["titre_duree"] > 210)
)
df = df[~condi].reset_index(drop='index')
df

plt.figure(figsize=(10, 6))
plt.boxplot(
    df["titre_duree"],
    patch_artist=True
)
plt.ylabel("Durée")
plt.title("Durée des films en minutes")
plt.tight_layout()
plt.show()

In [ ]:
total_genres = df.explode('titre_genres')['titre_genres'].value_counts()

plt.figure(figsize=(10, 8))
plt.barh(
    total_genres.index,
    total_genres.values,
    color='royalblue',
    edgecolor='black'
)
plt.title('Répartition des genres de films', fontsize=16)
plt.xlabel('Nombre de films', fontsize=14)
plt.ylabel('Genres', fontsize=14)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
total = df.explode('production_countries')['production_countries'].value_counts()[:10]

plt.figure(figsize=(16, 8))
plt.barh(
    total.index,
    total.values,
    color='royalblue',
    edgecolor='black'
)
plt.title('Nombre de films par Pays', fontsize=16)
plt.xlabel('Nombre de films', fontsize=14)
plt.ylabel('Pays', fontsize=14)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
boxplotdf = df

boxplotdf["cuts"] = boxplotdf["cuts"].astype(str)
boxplotdf.sort_values("cuts", inplace=True)

fig = px.box(
    data_frame = boxplotdf,
    y = "titre_duree",
    color = "cuts"
)

fig.show()

In [ ]:
df.to_parquet("clean_datasets/movies_cleaned.parquet")

In [ ]:
df.shape

In [ ]:
# # On filtre les films qui sont déja sortie au cinéma
# year_now = datetime.now().year

# condi = (
#     (df["person_job"].str.contains("act")) &
#     (df["titre_date_sortie"] <= year_now)
# )
# df = df[condi]
# actor = df.copy()

# """
# La moyenne d'age des acteurs a la date de sortie du film.
# Nous calculons la mediane sur cette base.

# other KPI, calcul la moyenne des ages en fonction de l'année

# """
# # ajouter colonne age pour les personnes
# actor["person_age"] = np.where(
#     (actor["person_birthdate"] > 1800) &
#     (actor["titre_date_sortie"] > 1800) &
#     (actor["titre_date_sortie"] - actor["person_birthdate"] >= 0),
#     (actor["titre_date_sortie"] - actor["person_birthdate"]),
#     0
# )
# actor.tail()

In [ ]:
# """
# imagine un fonction, qui prendrais en argument une liste de date de
# titre de sortie et retourne la moyenne d'age de ces années des acteurs.

# # Quels sont les acteurs les plus présents ?
# # À quelle période ?
# La durée moyenne des films s'allonge ou se raccourcit avec les années ?
# Les acteurs de série sont-ils les mêmes qu'au cinéma ?
# # Les acteurs ont en moyenne quel âge ?
# Quels sont les films les mieux notés ?
# Partagent-ils des caractéristiques communes ? etc...

# """

# bins, names = bins_generator(actor["titre_date_sortie"].max())

# actor["cuts"] = pd.cut(
#     actor["titre_date_sortie"],
#     bins=bins,
#     labels=names
# )

# actor_median = []
# for date in names[1:]:
#     condi = (
#         (actor["person_age"] != 0) &
#         (actor["cuts"] == date)
#     )
#     # print(f"Mediane for person_age {date} :", actor["person_age"][condi].median())
#     actor_median.append(actor["person_age"][condi].median())

# plt.figure(figsize=(10,6))
# plt.bar(names[1:], actor_median, color="royalblue")
# plt.xlabel("Années des film")
# plt.ylabel("Mediane")
# plt.title("Mediane acteur vs année")
# plt.xticks(rotation=45)
# plt.tight_layout()

In [ ]:
# # Top 5 des acteurs all time !
# print(actor["person_name"].value_counts().head(5))
# # Bottom 5 des acteurs all time !
# print(actor["person_name"].value_counts().tail(5))

In [ ]:
# # Top 5 actors by years
# for date in names[1:]:
#     top5 = actor['person_name'][condi].value_counts().head(5)
#     condi = (
#         (actor["cuts"] == date)
#     )
#     top5_str = '\n'.join(str(top5).split('\n')[:-1])

#     print(f"Top 5 actors by movies for {date}\n{'-'*20}\n{top5_str}")
#     print("-"*20)


In [ ]:
# #  World record of the longest documentary
# # source :https://fr.wikipedia.org/wiki/Ambianc%C3%A9
# longest = actor[actor["titre_duree"] == 43200]
# n = longest[["titre_str", "titre_duree"]].iloc[0]
# heure = n["titre_duree"] // 60
# jours = heure // 24
# print(f"Le film le plus long est {n['titre_str']} d'une durée de {heure} heures soit {jours} jours")
# # All time median for duration
# print(f"All time duration median : {round(actor['titre_duree'].median())} minutes")

In [ ]:
# # Median duration for movies between years.
# nodup = actor[~actor["titre_str"].duplicated(keep="first")]

# for date in names[1:]:
#     condi = (
#         (nodup["cuts"] == date)
#     )
#     med = round(nodup["titre_duree"][condi].median())
#     print(f"Median durations for movies {date}\nMedian for this years is {med} minutes")
#     print("-"*20)

In [ ]:
# print(len(yy[yy["titre_date_sortie"].isna()]))
# yy["titre_date_sortie"].fillna(1678, inplace=True)
# yy["person_birthdate"] = yy["person_birthdate"].astype(int).replace(0, 1678)
# condi = (
#     yy["person_birthdate"] < 1678
# )

# yy.loc[condi, "person_birthdate"] = 1678
# datime_col = ["titre_date_sortie", "person_birthdate"]
# for time in datime_col:
#     yy[time] = pd.to_datetime(yy[time], format="%Y")
# yy["titre_date_sortie"] = yy["titre_date_sortie"].dt.year